<a href="https://colab.research.google.com/github/shibanisankpal/datascience-and-analytics/blob/main/Credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predicting Credit Card Fraud using Logistic Regression

In [42]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

In [4]:
credit_card_data = pd.read_csv('/content/creditcard.csv')

In [5]:
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [6]:
credit_card_data.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
37717,39076,1.117749,-1.544665,-0.145357,-1.307482,-1.392833,-0.630833,-0.563659,-0.038716,-2.173892,1.561434,1.375139,-1.061420,-1.753850,0.635604,0.078165,-0.695348,0.876914,-0.299744,-0.153620,-0.155026,-0.334155,-1.108220,0.070461,0.115485,0.078726,-0.460844,-0.029143,0.029028,183.19,0.0
37718,39076,0.758729,-0.483875,0.125488,1.449159,-0.639231,-0.805011,0.406994,-0.221212,0.068777,-0.027164,-0.540305,-0.211977,-0.651315,0.529539,1.048556,0.216319,-0.423403,0.070802,-0.548487,0.311815,0.210712,0.063091,-0.316636,0.394472,0.558720,-0.304812,-0.033008,0.063331,250.00,0.0
37719,39076,-0.474907,0.835804,0.075878,-1.251283,2.492736,3.385419,0.169507,0.934749,-0.584224,-0.286547,-0.195320,-0.268205,-0.062458,0.307645,1.036639,0.500993,-0.881358,0.111643,0.426545,0.226808,-0.270398,-0.866353,-0.085720,0.986274,-0.082934,0.084198,0.310936,0.170342,1.49,0.0
37720,39077,-2.682780,-2.028562,1.745223,-1.157952,-0.143692,-0.807503,-1.154141,0.436917,-0.293442,0.078251,-0.691762,-0.753092,-0.287884,-0.617361,0.121762,1.247606,0.393749,-1.339760,-0.568519,0.103584,0.353396,0.968145,-0.776459,0.182662,-0.000427,-0.209058,0.256991,-0.585781,102.00,0.0
37721,39077,1.106292,-0.525727,0.926440,0.817696,-1.122177,-0.062703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37722 entries, 0 to 37721
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    37722 non-null  int64  
 1   V1      37722 non-null  float64
 2   V2      37722 non-null  float64
 3   V3      37722 non-null  float64
 4   V4      37722 non-null  float64
 5   V5      37722 non-null  float64
 6   V6      37722 non-null  float64
 7   V7      37721 non-null  float64
 8   V8      37721 non-null  float64
 9   V9      37721 non-null  float64
 10  V10     37721 non-null  float64
 11  V11     37721 non-null  float64
 12  V12     37721 non-null  float64
 13  V13     37721 non-null  float64
 14  V14     37721 non-null  float64
 15  V15     37721 non-null  float64
 16  V16     37721 non-null  float64
 17  V17     37721 non-null  float64
 18  V18     37721 non-null  float64
 19  V19     37721 non-null  float64
 20  V20     37721 non-null  float64
 21  V21     37721 non-null  float64
 22

In [8]:
# checking the number of missing values in each column
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        1
V8        1
V9        1
V10       1
V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [9]:
# distribution of legit transactions & fraudulent transactions
credit_card_data['Class'].value_counts()

0.0    37618
1.0      103
Name: Class, dtype: int64

0.0 -- Normal Transaction

1.0 -- Fraudulent Transaction

In [10]:
# separating the data
normal = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [11]:
print(normal.shape)
print(fraud.shape)

(37618, 31)
(103, 31)


In [13]:
fraud.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.000000,103.0
mean,20498.291262,-7.762676,5.838647,-10.957218,5.911555,-5.453798,-2.306085,-7.743100,3.867920,-2.961800,-6.700936,5.501599,-8.404377,0.413922,-8.354800,-0.029473,-4.949284,-8.060624,-2.865933,0.477900,0.652319,0.628539,-0.354537,-0.362138,-0.234731,0.300023,0.180390,0.822304,0.102481,90.471165,1.0
std,10224.604576,9.408942,4.799564,9.670659,2.851973,6.964496,1.617578,7.153267,6.233428,2.268143,4.128218,3.069263,4.418191,0.889796,4.492785,0.795429,3.543596,6.316090,2.654066,1.474923,0.906939,1.304033,0.916843,2.110441,0.520297,1.197724,0.379063,1.106307,0.656762,247.173335,0.0
min,406.000000,-30.552380,-5.198360,-31.103685,0.817253,-22.105532,-4.977692,-21.922811,-3.277535,-7.175097,-14.166795,-0.414575,-17.769143,-2.648076,-19.214325,-2.086989,-12.227189,-18.587366,-8.061208,-3.286150,-3.042626,-2.475962,-2.288686,-19.254328,-1.632653,-4.781606,-0.657488,-3.908080,-1.869290,0.000000,1.0
25%,11086.000000,-15.609012,2.536706,-20.883401,3.816369,-9.646293,-3.337874,-15.535968,0.132350,-4.266357,-8.984526,3.555128,-10.872992,-0.019215,-10.712525,-0.419534,-7.551854,-13.080863,-4.893749,-0.456116,0.234694,0.184108,-0.911406,-0.660823,-0.573174,-0.328146,-0.209664,0.291864,0.044583,1.000000,1.0
50%,20451.000000,-3.499108,4.140867,-6.240697,6.038515,-2.726961,-2.332285,-5.283435,1.202270,-3.565738,-6.810813,4.901683,-7.839539,0.543068,-7.891909,-0.054434,-5.367775,-10.961410,-4.480413,0.650807,0.576656,0.667927,-0.278485,-0.142812,-0.067195,0.185751,0.271139,0.737040,0.308205,3.760000,1.0
75%,28641.500000,-0.360399,8.921805,-3.445622,6.600628,-0.293699,-1.239059,-1.660418,4.129892,-1.175255,-3.684438,7.087329,-5.921868,1.042881,-5.455295,0.470263,-2.370139,-3.377071,-0.472371,1.296629,1.472367,1.757763,0.255747,0.303033,0.143699,1.336526,0.521196,1.579706,0.483431,99.990000,1.0
max,37167.000000,1.234235,16.713389,2.041003,11.927512,3.738853,3.628382,5.431271,20.007208,2.820829,3.245086,12.018913,1.237773,1.733917,1.500794,2.000635,2.581851,6.739384,3.042493,3.623332,2.102339,2.839596,1.745315,1.752227,0.645545,2.208209,0.816762,3.052358,1.108933,1809.680000,1.0


In [15]:
#grouping transactions and comparing averages
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,24778.212797,-0.186787,0.044196,0.749331,0.176750,-0.209569,0.105912,-0.092989,0.025693,0.243321,-0.061152,0.424604,-0.479428,0.271560,0.275963,0.090902,0.010825,0.179736,-0.081284,-0.036277,0.044683,-0.032046,-0.111604,-0.040404,0.007159,0.135739,0.021382,0.007030,0.003658,86.017411
1.0,20498.291262,-7.762676,5.838647,-10.957218,5.911555,-5.453798,-2.306085,-7.743100,3.867920,-2.961800,-6.700936,5.501599,-8.404377,0.413922,-8.354800,-0.029473,-4.949284,-8.060624,-2.865933,0.477900,0.652319,0.628539,-0.354537,-0.362138,-0.234731,0.300023,0.180390,0.822304,0.102481,90.471165


Number of fraudulent transactions -- 103

In [16]:
normal_sample = normal.sample(n=103)

In [20]:
new_dataset = pd.concat([normal_sample, fraud], axis=0)

In [18]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
20820,31286,1.251400,0.068772,0.728277,0.527328,-0.563445,-0.437373,-0.313458,-0.061477,0.346858,-0.097781,-0.685768,0.294659,0.706648,-0.061217,1.298802,0.699254,-0.748913,-0.022184,-0.141199,-0.034129,-0.111481,-0.301649,0.021561,-0.095621,0.240199,0.242782,-0.004912,0.023096,10.28,0.0
1569,1228,-0.693097,0.720897,0.487926,1.545283,-0.123343,0.151906,1.821822,-0.176592,-1.514396,-0.581211,1.491590,0.523781,0.824358,-0.591794,0.879726,-0.158535,0.702919,1.172998,1.713201,0.846036,0.200782,0.193611,0.288196,-0.081502,0.281742,-0.136080,0.050083,0.147487,279.93,0.0
28561,35054,-0.441388,1.076247,1.704878,0.043286,-0.208048,-1.067222,0.756704,-0.112600,-0.482501,-0.328670,-0.036218,0.277616,0.389582,0.079522,0.778803,0.084593,-0.298450,-0.508296,-0.113218,0.121837,-0.199346,-0.500683,0.023200,0.703272,-0.221953,0.049439,0.267014,0.125474,7.13,0.0
18497,29546,-9.151573,5.455072,-10.085887,2.031553,-5.183587,-1.763760,-3.630099,6.248238,0.125275,-0.442365,-1.582931,1.001587,-0.589491,0.610694,1.411001,2.625975,5.372306,1.730982,-1.149257,-0.028461,-0.086822,-1.064605,-0.413784,-0.752004,0.255406,-0.309974,-0.038259,-0.312459,99.99,0.0
15766,27203,-0.579139,0.520783,1.482508,1.105415,0.530343,-0.531251,0.130093,0.065134,-0.737187,-0.301009,0.111571,-0.125566,0.147972,-0.187950,2.095363,-0.564654,0.919389,-0.333191,1.249668,0.309483,-0.110056,-0.415899,0.130204,0.044928,-0.584096,0.328311,0.117866,0.153043,10.99,0.0


In [22]:
new_dataset['Class'].value_counts()

1.0    103
0.0    103
Name: Class, dtype: int64

In [23]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,24828.058252,-0.016850,0.110740,0.670276,0.162410,-0.270081,0.101027,-0.088687,0.038742,0.080717,0.018685,0.508827,-0.427294,0.305424,0.289582,0.348384,0.082079,0.144790,0.030764,0.005304,0.018229,-0.112225,-0.133755,-0.119945,0.010184,0.070738,0.11342,0.053813,-0.036332,80.650097
1.0,20498.291262,-7.762676,5.838647,-10.957218,5.911555,-5.453798,-2.306085,-7.743100,3.867920,-2.961800,-6.700936,5.501599,-8.404377,0.413922,-8.354800,-0.029473,-4.949284,-8.060624,-2.865933,0.477900,0.652319,0.628539,-0.354537,-0.362138,-0.234731,0.300023,0.18039,0.822304,0.102481,90.471165


In [24]:
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

In [25]:
print(X)

        Time        V1        V2  ...       V27       V28  Amount
20820  31286  1.251400  0.068772  ... -0.004912  0.023096   10.28
1569    1228 -0.693097  0.720897  ...  0.050083  0.147487  279.93
28561  35054 -0.441388  1.076247  ...  0.267014  0.125474    7.13
18497  29546 -9.151573  5.455072  ... -0.038259 -0.312459   99.99
15766  27203 -0.579139  0.520783  ...  0.117866  0.153043   10.99
...      ...       ...       ...  ...       ...       ...     ...
30442  35926 -3.896583  4.518355  ...  0.635789  0.501050    4.56
30473  35942 -4.194074  4.382897  ...  0.473934  0.473757   14.46
30496  35953 -4.844372  5.649439  ...  0.210214  0.391855  111.70
31002  36170 -5.685013  5.776516  ...  0.303445  0.219380  111.70
33276  37167 -7.923891 -5.198360  ...  1.937421 -1.552593   12.31

[206 rows x 30 columns]


In [26]:
print(Y)

20820    0.0
1569     0.0
28561    0.0
18497    0.0
15766    0.0
        ... 
30442    1.0
30473    1.0
30496    1.0
31002    1.0
33276    1.0
Name: Class, Length: 206, dtype: float64


**Splitting into training and testing data**

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, test_size = 0.2, random_state= 2)

In [40]:
print(X.shape, X_train.shape, X_test.shape)

(206, 30) (164, 30) (42, 30)


In [53]:
X_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
18773,29753,0.269614,3.549755,-5.810353,5.809370,1.538808,-2.269219,-0.824203,0.351070,-3.759059,-4.592390,5.479392,-5.657858,-2.648076,-10.384890,-0.225946,-0.701883,0.409736,0.939604,-1.954300,0.310525,0.371121,-0.322290,-0.549856,-0.520629,1.378210,0.564714,0.553255,0.402400,0.68
20943,31368,-1.204027,-0.301587,1.528184,1.089747,0.865967,-0.268666,1.257466,-0.391826,-0.651506,0.451684,1.855183,0.424442,-0.474791,0.255632,0.693968,-0.436524,-0.323221,-0.424243,0.593831,-0.066301,-0.418318,-0.658346,1.102784,0.128730,-0.661771,-0.873500,-0.109485,-0.409300,99.00
8615,11629,-3.891192,7.098916,-11.426467,8.607557,-2.065706,-2.985288,-8.138589,2.973928,-6.272790,-13.193415,11.619723,-17.631606,-0.355194,-18.822087,-1.283129,-10.031097,-15.227008,-5.322009,-0.501751,1.382619,1.757085,-0.189709,-0.508629,-1.189308,1.188536,0.605242,1.881529,0.875260,1.00
6903,8886,-2.535852,5.793644,-7.618463,6.395830,-0.065210,-3.136372,-3.104557,1.823233,-3.878658,-7.297803,7.071669,-10.001046,-0.207935,-10.860698,-1.690825,-2.360857,-2.760097,0.298103,-1.385558,0.408704,0.716720,-0.448060,-0.402407,-0.288835,1.011752,0.425965,0.413140,0.308205,1.00
18382,29435,-0.881525,-0.224662,1.171335,-1.950666,0.325812,-0.259574,0.970046,-0.165940,-1.752909,0.353449,1.440984,0.080801,0.544231,-0.210013,-0.568526,0.985359,-0.118352,-1.719919,0.255039,0.635894,-0.058447,-0.367556,0.222008,-0.342237,0.045287,-0.666547,0.091861,-0.063773,135.00


**Logistic Regression**

In [43]:
lr = LogisticRegression()
lr.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
y_pred = lr.predict(X_test)
print(y_pred)

[0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 0. 0.
 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0.]


In [55]:
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(lr.score(X_test, Y_test)))

Accuracy of logistic regression classifier on test set: 0.90


**Confusion matrix**

In [57]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(Y_test, y_pred)
print(confusion_matrix)

[[20  1]
 [ 3 18]]


The result is telling us that we have 20+18=38 correct predictions and 3+1=4 incorrect predictions.